In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as dsets
from torchvision import models
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/thesis06_20w_1s')


from tqdm import tqdm

from flows import PlanarFlow
from utils import Binarize
from codes import Linear_flipout, Flatten, count_parameters, EfficientNet

from torchmeta.datasets import Omniglot, CIFARFS
from torchmeta.transforms import Categorical, ClassSplitter, Rotation
from torchvision.transforms import Compose, Resize, ToTensor
from torchmeta.utils.data import BatchMetaDataLoader


#from __future__ import print_function
import argparse
import cv2
import matplotlib.pyplot as plt

import os
cur_dir = "C:/Users/KJH/OneDrive - skku.edu/KJH/Projects/2019winter_research"
#cur_dir = "C:/Users/KJH-Laptop/OneDrive - skku.edu/KJH/Projects/2019winter_research/"
os.chdir(cur_dir)
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
import copy
import random as rd

device = torch.device('cuda')

class net(nn.Module):
    def __init__(self, num_classes):
        super(net, self).__init__()
        self.input_dim = [1, 28, 28]
        self.num_classes = num_classes
        
        self.ctx = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=True)
        self.ctx.conv_stem = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        self.ctx.classifier = nn.Sequential(
            nn.BatchNorm1d(1280),
            nn.Linear(1280, 320, bias = True),
            nn.ReLU()
        )
        
        self.layer = nn.LSTM(input_size=1280, hidden_size=1280, num_layers=5, bias=True, batch_first = True).to(device)
        self.dec_mu = nn.ModuleList()
        self.dec_logvar = nn.ModuleList()
        
        self.num_params = [[200 * 200, 200], [200 * 200, 200], [200 * self.num_classes, self.num_classes]]
        
        for layer_size in self.num_params:
            self.dec_mu.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
            
            self.dec_logvar.append(
                nn.Sequential(
                    nn.ELU(),
                    nn.Linear(1280, 1280, bias = True),
                    nn.ELU(),
                    nn.Linear(1280, layer_size[0] + layer_size[1], bias = True),
                ).to(device))
        
        self.encoder = nn.Sequential(
            nn.BatchNorm2d(1),
            Flatten(),            
            nn.Linear(784, 200, bias = False),
            nn.ELU()
        )

            
            
    def forward(self, input_train, label_train, input_test, label_test, adapt_lr, adapt_step = 1):
        ctx = self.ctx(input_train).view(input_train.shape[0], 5, -1)
        h = torch.stack([ctx[torch.where(label_train == x)].mean(dim = 0) for x in range(self.num_classes)], dim = 1)
        h = h.view(5, 1, -1)
        
        params = nn.ParameterList()
        param_vals = []
        kld = torch.tensor(0., device=device)
        c = torch.empty([5, 1, 1280], requires_grad = False, device = device).fill_(0)
        x_test_init = self.encoder(input_test)
        x_test = x_test_init

        for ind, (dec_mu, dec_logvar) in enumerate(zip(self.dec_mu, self.dec_logvar)):
            x = torch.empty([1, 1, 1280], requires_grad=False, device = device).normal_(0, 1)
            f, (h, c) = self.layer(x, (h, c))
            
            mu = dec_mu(f.view(1, -1)).squeeze()
            logvar = dec_logvar(f.view(1, -1)).squeeze()
            params.append(nn.Parameter(torch.stack((mu, logvar), dim = 0), requires_grad = True))
        
        optimizer = optim.SGD(params, lr = adapt_lr)
        
        for step in range(adapt_step + 1):
            x_test = x_test_init
            kld = torch.tensor(0., device=device)
            for ind, param in enumerate(params):
                param.retain_grad()

                weight_mu = param[0, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_mu = param[0, self.num_params[ind][0]:]

                weight_logvar = param[1, :self.num_params[ind][0]].view(-1, self.num_params[ind][1])
                bias_logvar = param[1, self.num_params[ind][0]:]

                weight_noise = torch.empty(weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
                bias_noise = torch.empty(bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
                in_sign = torch.empty(x_test.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
                out_sign = torch.empty([x_test.shape[0], self.num_params[ind][1]], requires_grad = False, device = device).uniform_(-1,1).sign()

                x_test = torch.mm(x_test, weight_mu) + torch.mm(in_sign * x_test, weight_noise * weight_mu * weight_logvar.div(2).exp()) * out_sign
                x_test += (1 + bias_noise * bias_logvar.div(2).exp()) * bias_mu
                x_test = F.elu(x_test)

                kld += (mu.pow(2) - logvar + logvar.exp() - 1).mean()/2
                
            if step < adapt_step:
                optimizer.zero_grad()
                loss = F.cross_entropy(x_test, label_test) + 1e-6 * kld
                loss.backward(retain_graph = True)
                optimizer.step()

        return x_test, kld

In [2]:
batch_size = 16
meta_trainset = Omniglot('./data/',
                   # Number of ways
                   num_classes_per_task=20,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=20),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_train=True,
                   download=True)
meta_trainset = ClassSplitter(meta_trainset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_trainloader = BatchMetaDataLoader(meta_trainset, batch_size=batch_size, num_workers=0)

model = net(20).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, test_target, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 20 * 15
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

Using cache found in C:\Users\KJH/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


  0) loss = 2.603963, kld = 0.000764, acc = 0.237708, time = 17.765 sec
 10) loss = 2.070702, kld = 0.000124, acc = 0.426667, time = 30.516 sec
 20) loss = 2.021549, kld = 0.000041, acc = 0.460417, time = 30.990 sec
 30) loss = 1.873713, kld = 0.000021, acc = 0.499583, time = 30.326 sec
 40) loss = 1.851576, kld = 0.000015, acc = 0.499167, time = 31.190 sec
 50) loss = 1.920723, kld = 0.000012, acc = 0.500208, time = 30.166 sec
 60) loss = 1.933874, kld = 0.000011, acc = 0.482083, time = 30.712 sec
 70) loss = 1.856399, kld = 0.000011, acc = 0.501458, time = 30.725 sec
 80) loss = 2.082198, kld = 0.000011, acc = 0.444167, time = 30.432 sec
 90) loss = 1.875944, kld = 0.000011, acc = 0.501875, time = 31.062 sec
100) loss = 1.768155, kld = 0.000010, acc = 0.526458, time = 30.601 sec
110) loss = 1.845184, kld = 0.000010, acc = 0.513542, time = 30.124 sec
120) loss = 1.921565, kld = 0.000010, acc = 0.503333, time = 30.439 sec
130) loss = 1.740383, kld = 0.000011, acc = 0.531875, time = 31.

In [3]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, test_target, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 20 * 15
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 2.531750, kld = 0.000010, acc = 0.418125, time = 29.869 sec
 10) loss = 2.003087, kld = 0.000010, acc = 0.461875, time = 30.248 sec
 20) loss = 1.943420, kld = 0.000010, acc = 0.492083, time = 30.127 sec
 30) loss = 1.942439, kld = 0.000010, acc = 0.503750, time = 30.166 sec
 40) loss = 1.869648, kld = 0.000010, acc = 0.517083, time = 30.000 sec
 50) loss = 1.982632, kld = 0.000010, acc = 0.490833, time = 30.098 sec
 60) loss = 1.494840, kld = 0.000010, acc = 0.585417, time = 30.877 sec
 70) loss = 1.660182, kld = 0.000010, acc = 0.551667, time = 31.406 sec
 80) loss = 1.659267, kld = 0.000010, acc = 0.568750, time = 30.541 sec
 90) loss = 1.527660, kld = 0.000010, acc = 0.585417, time = 31.039 sec
100) loss = 1.698551, kld = 0.000010, acc = 0.542917, time = 31.140 sec
110) loss = 1.744369, kld = 0.000010, acc = 0.541458, time = 31.131 sec
120) loss = 2.010949, kld = 0.000010, acc = 0.481458, time = 30.920 sec
130) loss = 1.537902, kld = 0.000010, acc = 0.582292, time = 31.

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1e-6)

num_batches = 200
for batch_idx, meta_train_batch in zip(range(num_batches), meta_trainloader):
    start = time.time()

    train_inputs, train_targets = [x.to(device) for x in meta_train_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_train_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, test_target, 0.5, 5)
        loss = criterion(pred, test_target)
        (loss + 1e-6 * kld).backward()
        optimizer.step()
        with torch.no_grad():
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld
    cum_loss /= batch_size
    accuracy /= batch_size * 20 * 15
    reg /= batch_size

    if batch_idx % 10 == 0:
        print("%3d) loss = %f, kld = %f, acc = %f, time = %.3f sec" %(batch_idx, cum_loss, reg, accuracy, time.time() - start))

  0) loss = 1.651697, kld = 0.000010, acc = 0.552083, time = 30.315 sec
 10) loss = 1.846209, kld = 0.000010, acc = 0.525417, time = 31.014 sec
 20) loss = 1.680036, kld = 0.000010, acc = 0.565625, time = 31.535 sec
 30) loss = 1.593975, kld = 0.000010, acc = 0.561667, time = 31.352 sec
 40) loss = 1.688189, kld = 0.000010, acc = 0.541875, time = 30.849 sec
 50) loss = 1.697957, kld = 0.000010, acc = 0.551458, time = 30.337 sec
 60) loss = 1.623374, kld = 0.000010, acc = 0.561667, time = 30.434 sec
 70) loss = 1.761743, kld = 0.000010, acc = 0.541875, time = 30.296 sec
 80) loss = 1.679443, kld = 0.000010, acc = 0.565000, time = 30.201 sec
 90) loss = 1.729807, kld = 0.000010, acc = 0.535625, time = 30.275 sec
100) loss = 1.851496, kld = 0.000010, acc = 0.495625, time = 30.841 sec
110) loss = 1.534651, kld = 0.000010, acc = 0.571042, time = 30.097 sec
120) loss = 1.737582, kld = 0.000010, acc = 0.535833, time = 29.730 sec
130) loss = 1.667550, kld = 0.000010, acc = 0.544375, time = 30.

In [7]:
meta_testset  = Omniglot('./data/',
                   # Number of ways
                   num_classes_per_task=20,
                   # Resize the images to 28x28 and converts them to PyTorch tensors (from Torchvision)
                   transform=Compose([Resize(28), ToTensor()]),
                   # Transform the labels to integers (e.g. ("Glagolitic/character01", "Sanskrit/character14", ...) to (0, 1, ...))
                   target_transform=Categorical(num_classes=20),
                   # Creates new virtual classes with rotated versions of the images (from Santoro et al., 2016)
                   class_augmentations=[Rotation([90, 180, 270])],
                   meta_test=True,
                   download=True)
meta_testset = ClassSplitter(meta_testset, shuffle=True, num_train_per_class=5, num_test_per_class=15)
meta_testloader = BatchMetaDataLoader(meta_testset, batch_size=batch_size, num_workers=0)

tot_loss = torch.tensor(0., device=device)
tot_acc = torch.tensor(0., device=device)
tot_reg = torch.tensor(0., device=device)

for batch_idx, meta_test_batch in zip(range(num_batches), meta_testloader):
    start = time.time()
    train_inputs, train_targets = [x.to(device) for x in meta_test_batch["train"]]
    test_inputs, test_targets = [x.to(device) for x in meta_test_batch["test"]]
    
    cum_loss = torch.tensor(0., device=device)
    accuracy = torch.tensor(0., device=device)
    reg = torch.tensor(0., device=device)

    for task_idx, (train_input, train_target, test_input, test_target) in enumerate(
        zip(train_inputs, train_targets, test_inputs, test_targets)):
        optimizer.zero_grad()
        pred, kld = model(train_input, train_target, test_input, test_target, 0.5, 5)
        with torch.no_grad():            
            loss = criterion(pred, test_target)
            cum_loss += loss
            accuracy += torch.sum(pred.argmax(1) == test_target.cuda())
            reg += kld

    tot_loss += cum_loss / batch_size
    tot_acc += accuracy  / (batch_size * 20 * 15)
    tot_reg += reg / batch_size
        
tot_loss /= num_batches
tot_acc /= num_batches
tot_reg /= num_batches
        
print("loss = %f, test_kld = %f, meta_test_acc = %f, time = %.3f sec" %(tot_loss, tot_reg, tot_acc, time.time() - start))

loss = 1.743715, test_kld = 0.000010, meta_test_acc = 0.543837, time = 24.121 sec


In [8]:
torch.save(model.state_dict(), "./save/thesis06_omniglot20w5s_5step")